In [11]:
!pip install -r requirements.txt --no-warn-conflicts --progress-bar emoji
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import girder_client
import json
import os
import pandas as pd
import psycopg2
import postgres_to_girder
import requests
with open ("config.json", "r") as j:
    config = json.load(j)
with open ("context.json", "r") as j:
    context = json.load(j)

In [12]:
gc = girder_client.GirderClient(
    apiUrl="".join([
        "http://",
        config["girder"]["host"],
        "/api/v1"
    ])
)
gc.authenticate(
    config["girder"]["user"],
    config["girder"]["password"])

In [21]:
activities_id = postgres_to_girder.get_girder_id_by_name(
    entity="collection",
    name="activities",
    girder_connection=gc
)

None


In [14]:
activities_id

'5b1850bead97092bdb8e99d5'

In [15]:
try:
    conn = psycopg2.connect(
        " ".join(
            [
                "=".join([
                    key,
                    config["postgres"][key]
                ]) for key in config["postgres"]
            ]
        )
    )
except:
    print("I am unable to connect to the database")

In [16]:
acts = pd.io.sql.read_sql_query(
    "SELECT * FROM acts;",
    conn
)
users = pd.io.sql.read_sql_query(
    "SELECT * FROM users;",
    conn
)

In [27]:
for activity in set(acts["title"]):
    if not postgres_to_girder.get_girder_id_by_name(
        entity="folder",
        name=activity,
        girder_connection=gc,
        parent=(
            "collection",
            activities_id
        )
    ):
        gc.createFolder(
            name=activity,
            parentId=activities_id,
            parentType="collection",
            public=True,
            reuseExisting=True
        )